# `TensorFlow`/`Keras`

[Keras](https://keras.io/) is a high-level neural networks API, written in Python and capable of running on top of [TensorFlow](https://www.tensorflow.org/), [CNTK](https://docs.microsoft.com/de-de/cognitive-toolkit/), or [Theano](http://www.deeplearning.net/software/theano/). It was developed with a focus on enabling fast experimentation. *Being able to go from idea to result with the least possible delay is key to doing good research.*

**Note 1:** This is not an introduction to deep neural networks as this would explode the scope of this notebook. But we want to show you how you can implement a convoluted neural network to classify neuroimages, in our case fMRI images.  
**Note 2:** We want to thank [Anisha Keshavan](https://github.com/akeshavan) as a lot of the content in this notebook is coming from here [introduction notebook](http://nbviewer.jupyter.org/github/brainhack101/IntroDL/blob/master/IntroToKeras.ipynb) about Keras.

## Setup

In [ ]:
import warnings
warnings.filterwarnings("ignore")

from nilearn import plotting
%matplotlib inline
import numpy as np
import nibabel as nb
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="white")
import os
import datetime
import tensorflow as tf
import plotly.graph_objects as go
from plotly import figure_factory as ff

## Load machine learning dataset

We will load a dataset that was prepared to enable quick showcases/introductions for machine learning. It includes an `anatomical template image` (we will need this for visualization), as well as a `4D fMRI image` from a `resting state scan`. The dataset ist from Zang et al.. It contains 48 subjects, where each subject did two `resting-state fMRI` recordings. Once with eyes open and once with eyes closed. The data was already pre-processed and is already ready for the machine learning notebooks. Note: The data diverges from the original data in the way, that we only consider the first 100 volumes for this tutorial. The original dataset had 240 volumes per run.

<img src="data/resting_state_eyes.gif"/>


In [ ]:
anat = nb.load('data/MNI152_T1_1mm.nii.gz')
func = nb.load('data/dataset_ML.nii.gz')

Let's check how the `4D fMRI image` looks like via plotting its `mean over time`.

In [ ]:
from nilearn.image import mean_img
from nilearn.plotting import plot_anat

In [ ]:
plot_anat(mean_img(func), cmap='magma', colorbar=False, display_mode='x', vmax=2, annotate=False,
          cut_coords=range(0, 49, 12), title='Mean value of machine learning dataset');

# Specifying labels and chunks

As in every other `machine` or `deep learning` application, we need some chunks and label variables to train the `neural network`. The labels are important so that we can predict what we want to classify. And the chunks are just an easy way to make sure that the training and test dataset are split in an equal/balanced way.

So, as before, we specify again which volumes of the dataset were recorded during eyes **closed** resting state and which ones were recorded during eyes **open** resting state recording.

From the dataset release we know that we have a total of `384 volumes` in our `dataset_ML.nii.gz` file and that it's always `4 volumes` of the condition `eyes closed`, followed by `4 volumes` of the condition `eyes open`, etc. Therefore our labels should be as follows:

In [ ]:
labels = np.ravel([[['closed'] * 4, ['open'] * 4] for i in range(48)])
labels[:20]

***Second***, the `chunks` variable should not switch between subjects. So, as before, we can again specify 6 chunks of 64 volumes (8 subjects), each:

In [ ]:
chunks = np.ravel([[i] * 64 for i in range(6)])
chunks[:150]

# Keras - 2D Example

Convoluted neural networks are very powerful (as you will see), but the computation power to train the models can be incredibly demanding. For this reason, it's sometimes recommended to try to reduce the input space if possible.

In our case, we could try to not train the neural network only on one very thin slab (a few slices) of the brain. So, instead of taking the data matrix of the whole brain, we just take 2 slices in the region that we think is most likely to be predictive for the question at hand.

We know (or suspect) that the regions with the most predictive power are probably somewhere around the eyes and in the visual cortex. So let's try to specify a few slices that cover those regions.

So, let's try to just take a few slices around the eyes:

In [ ]:
plot_anat(mean_img(func).slicer[...,5:-25], cmap='magma', colorbar=False,
          display_mode='x', vmax=2, annotate=False, cut_coords=range(0, 49, 12),
          title='Slab of the machine learning mean image');

Hmm... That doesn't seem to work. We want to cover the eyes and the visual cortex. Like this, we're too far down in the back of the head (at the Cerebellum). One solution to this is to rotate the volume.

So let's do that:

In [ ]:
# Rotation parameters
phi = 0.35
cos = np.cos(phi)
sin = np.sin(phi)

# Compute rotation matrix around x-axis
rotation_affine = np.array([[1, 0, 0, 0],
                            [0, cos, -sin, 0],
                            [0, sin, cos, 0],
                            [0, 0, 0, 1]])
new_affine = rotation_affine.dot(func.affine)

In [ ]:
# Rotate and resample image to new orientation
from nilearn.image import resample_img
new_img = nb.Nifti1Image(func.get_fdata(), new_affine)
img_rot = resample_img(new_img, func.affine, interpolation='continuous')
del func
del new_img

In [ ]:
# Delete zero-only rows and columns
from nilearn.image import crop_img
img_crop = crop_img(img_rot)
del img_rot

Let's check if the rotation worked.

In [ ]:
plot_anat(mean_img(img_crop), cmap='magma', colorbar=False, display_mode='x', vmax=2, annotate=False,
          cut_coords=range(-20, 30, 12), title='Rotated machine learning dataset');

Perfect! And which slab should we take? Let's try the slices 12, 13 and 14.

In [ ]:
from nilearn.plotting import plot_stat_map
img_slab = img_crop.slicer[..., 12:15, :]
plot_stat_map(mean_img(img_slab), cmap='magma', bg_img=mean_img(img_crop), colorbar=False,
              display_mode='x', vmax=2, annotate=False, cut_coords=range(-20, 30, 12),
              title='Slab of rotated machine learning dataset');

Perfect, the slab seems to contain exactly what we want. Now that the data is ready we can continue with the actual machine learning part.

## Split data into a training and testing set

First things first, we need to define a training and testing set. This is *really* important because we need to make sure that our model can generalize to new, unseen data. Here, we randomly shuffle our data, and reserve 80% of it for our training data, and the remaining 20% for testing.

So let's first get the data in the right structure for keras. For this, we need to swap some of the dimensions of our data matrix.

In [ ]:
data = np.rollaxis(img_slab.get_fdata(), 3, 0)
data.shape

As you can see, the goal is to have in the first dimension, the different volumes, and then the volume itself. Keep in mind, that the last dimension (here of size 2), are considered as `channels` in the keras model that we will be using below.

**Note:** To make this notebook reproducible, i.e. always leading to the "same" results. Let's set a seed point for the random split of the dataset. This should only be done for teaching purposes, but not for real research as randomness and chance are a crucial part of machine learning.

In [ ]:
from numpy.random import seed
seed(0)

As a next step, let's create a index list that we can use to split the data and labels into training and test sets:

In [ ]:
# Create list of indices and shuffle them
N = data.shape[0]
indices = np.arange(N)
np.random.shuffle(indices)

#  Cut the dataset at 80% to create the training and test set
N_80p = int(0.8 * N)
indices_train = indices[:N_80p]
indices_test = indices[N_80p:]

# Split the data into training and test sets
X_train = data[indices_train, ...]
X_test = data[indices_test, ...]

print(X_train.shape, X_test.shape)

## Create outcome variable

We need to define a variable that holds the outcome variable (1 or 0) that indicates whether or not the resting-state images were recorded with eyes opened or closed. Luckily we have this information already stored in the `labels` variable above. So let's split these labels in training and test set:

In [ ]:
y_train = labels[indices_train] == 'open'
y_test = labels[indices_test] == 'open'

## Data Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
scaler = StandardScaler()
pca = PCA()
tsne = TSNE()

In [ ]:
X_scaled = scaler.fit_transform(X_train.reshape(len(X_train), -1))

In [ ]:
X_pca = pca.fit_transform(X_scaled)

In [ ]:
plt.plot(pca.explained_variance_ratio_.cumsum())

In [ ]:
y_train

In [ ]:
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_train, cmap='bwr')

In [ ]:
X_tsne = tsne.fit_transform(X_pca)

In [ ]:
plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=y_train, cmap='bwr')

In [ ]:
mean = X_train.mean(axis=0)
mean.shape

In [ ]:
std = X_train.std(axis=0)
std.shape

In [ ]:
plt.hist(np.ravel(std), bins=100);
plt.vlines(0.05, 0, 1000, colors='red')

In [ ]:
std[std<0.05] = 0

In [ ]:
plt.hist(np.ravel(mean), bins=100);
plt.vlines(0.25, 0, 1000, colors='red')

In [ ]:
mean[mean<0.05] = 0

In [ ]:
mask = (mean*std)!=0

In [ ]:
X_zscore_tr = (X_train-mean)/std
X_zscore_te = (X_test-mean)/std
X_zscore_tr.shape

In [ ]:
X_zscore_tr[np.isnan(X_zscore_tr)]=0
X_zscore_te[np.isnan(X_zscore_te)]=0

In [ ]:
X_zscore_tr[np.isinf(X_zscore_tr)]=0
X_zscore_te[np.isinf(X_zscore_te)]=0

And now we're good to go.

## Creating a Sequential Model

Now come the fun and tricky part. We need to specify the structure of our convoluted neural network. As a quick reminder, a convoluted neural network consists of some convolution layers, pooling layers, some flattening layers and some full connect layers:

<img src="data/deep_neural_networks.png"/>

Taken from: https://www.mathworks.com/videos/introduction-to-deep-learning-what-are-convolutional-neural-networks--1489512765771.html

So as a first step, let's import all modules that we need to create the keras model:

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AvgPool2D, BatchNormalization
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam, SGD

As a next step, we should specify some of the model parameters that we want to be identical throughout the model:

In [ ]:
# Get shape of input data
data_shape = tuple(X_train.shape[1:])

# Specify shape of convolution kernel
kernel_size = (3, 3)

# Specify number of output categories
n_classes = 2

Now comes the big part... the model, i.e. the structure of the neural network! We want to make clear that we're no experts in deep neural networks and therefore, the model below might not necessarily be a good model. But we chose it as it can be rather quickly estimated and has rather few parameters to estimate.

In [ ]:
# Specify number of filters per layer
filters = 32

model = Sequential()

model.add(Conv2D(filters, kernel_size, activation='relu', input_shape=data_shape))
model.add(BatchNormalization())
model.add(MaxPooling2D())
filters *= 2

model.add(Conv2D(filters, kernel_size, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D())
filters *= 2

model.add(Conv2D(filters, kernel_size, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D())
filters *= 2

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam', # swap out for sgd 
              metrics=['accuracy'])

model.summary()

That's what our model looks like! Cool!

## Fitting the Model

The next step is now, of course, to fit our model to the training data. In our case we have two parameters that we can work with:

*First*: How many iterations of the model fitting should be computed

In [ ]:
nEpochs = 100  # Increase this value for better results (i.e., more training)

*Second*: How many elements (volumes) should be considered at once for the updating of the weights?

In [ ]:
batch_size = 32   # Increasing this value might speed up fitting

We will also define a `log directory` so that we can `evaluate` our `model` later on as best as possible. 

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

So let's test the model:

In [ ]:
%time fit = model.fit(X_zscore_tr, y_train, epochs=nEpochs, batch_size=batch_size,\
                      validation_split=0.2, callbacks=[tensorboard_callback])

## Performance during model fitting

Let's take a look at the `loss` and `accuracy values` during the different `epochs`, starting with `accuracy values`:

In [ ]:
fig = plt.figure(figsize=(10, 4))
epoch = np.arange(nEpochs) + 1
fontsize = 16
plt.plot(epoch, fit.history['acc'], marker="o", linewidth=2,
         color="steelblue", label="acc")
plt.plot(epoch, fit.history['val_acc'], marker="o", linewidth=2,
         color="orange", label="val_acc")
plt.xlabel('epoch', fontsize=fontsize)
plt.xticks(fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.legend(frameon=False, fontsize=16);

Given that we are running this interactively in a `jupyter notebook`, we can make use of its capabilities and create an interactive graph using `plotly`:

In [ ]:
def accuracy_epoch_plotly():

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=epoch, y=fit.history['acc'],
                        mode='lines+markers',
                        name='acc'))
    fig.add_trace(go.Scatter(x=epoch, y=fit.history['val_acc'],
                        mode='lines+markers',
                        name='val_acc'))

    fig.update_layout(
       title={
            'text': "Accuracy per epoch",
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        xaxis_title="Epoch",
        yaxis_title="Accuracy",
        legend_title="Type",
        template='plotly_white'
    )

    fig.show()

In [ ]:
accuracy_epoch_plotly()

Next, we check the `loss values`, at first via a `static plot`:

In [ ]:
fig = plt.figure(figsize=(10, 4))
epoch = np.arange(nEpochs) + 1
fontsize = 16
plt.plot(epoch, fit.history['loss'], marker="o", linewidth=2,
         color="steelblue", label="loss")
plt.plot(epoch, fit.history['val_loss'], marker="o", linewidth=2,
         color="orange", label="val_loss")
plt.xlabel('epoch', fontsize=fontsize)
plt.xticks(fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.legend(frameon=False, fontsize=16);

and second via an `interactive plot`:

In [ ]:
def loss_epoch_plotly():

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=epoch, y=fit.history['loss'],
                        mode='lines+markers',
                        name='loss'))
    fig.add_trace(go.Scatter(x=epoch, y=fit.history['val_loss'],
                        mode='lines+markers',
                        name='val_loss'))

    fig.update_layout(
       title={
            'text': "Loss per epoch",
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        xaxis_title="Epoch",
        yaxis_title="Loss",
        legend_title="Type",
        template='plotly_white'
    )

    fig.show()

In [ ]:
loss_epoch_plotly()

Great, it seems that accuracy is constantly increasing and the loss is continuing to drop. But how well is our model doing on the test data?

## Evaluating the model

In [ ]:
evaluation = model.evaluate(X_zscore_te, y_test)
print('Loss in Test set:      %.02f' % (evaluation[0]))
print('Accuracy in Test set:  %.02f' % (evaluation[1] * 100))

# Confusion Matrix

We can also evaluate the model in more detail via obtaining a `confusion matrix` which will provide as with more information concerning the `sensitivity` and `specificity` of our `model`. After getting the `predicted` and `true labels` 

In [ ]:
y_pred = np.argmax(model.predict(X_zscore_te), axis=1)
y_pred

In [ ]:
y_true = y_test * 1
y_true

We can compute the `confusion matrix` and plot it:

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd

In [ ]:
class_labels = ['closed', 'open']
cm = pd.DataFrame(confusion_matrix(y_true, y_pred), index=class_labels, columns=class_labels)
sns.heatmap(cm, square=True, annot=True);

Again, an `interactive plot` might be nice as well:

In [ ]:
def confusion_matrix_plotly():
    
    z_text = [[str(y) for y in x] for x in cm.to_numpy()]
    fig = ff.create_annotated_heatmap(cm.to_numpy(), x=class_labels, y=class_labels, annotation_text=z_text, colorscale='Magma')

    # add custom xaxis title
    fig.add_annotation(dict(font=dict(color="black",size=14),
                            x=0.5,
                            y=-0.15,
                            showarrow=False,
                            text="Predicted value",
                            xref="paper",
                            yref="paper"))

    # add custom yaxis title
    fig.add_annotation(dict(font=dict(color="black",size=14),
                            x=-0.1,
                            y=0.45,
                            showarrow=False,
                            text="Real value",
                            textangle=-90,
                            xref="paper",
                            yref="paper"))
    fig.show()

In [ ]:
confusion_matrix_plotly()

## Analyze prediction values

What are the predicted values of the test set?

In [ ]:
y_pred = model.predict(X_zscore_te)
y_pred[:10,:]

As you can see, those values can be between 0 and 1.

In [ ]:
fig = plt.figure(figsize=(6, 4))
fontsize = 16
plt.hist(y_pred[:,0], bins=16, label='eyes closed')
plt.hist(y_pred[:,1], bins=16, label='eyes open');
plt.xticks(fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.legend(frameon=False, fontsize=16);

As usual, we also generate an `interactive plot`:

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=y_pred[:,0],name='eyes closed', nbinsx=16, marker_color='blue'))
fig.add_trace(go.Histogram(x=y_pred[:,1],name='eyes open', nbinsx=16, marker_color='orange'))

fig.update_layout(barmode='stack', template='plotly_white')

fig.show()

The more both distributions are distributed around chance level, the weaker your model is.

**Note:** Keep in mind that we trained the whole model only on one split of test and training data. Ideally, you would repeat this process many times so that your results become less dependent on what kind of split you did.

## Visualizing Hidden Layers

Finally, as a cool additional feature: We can now visualize the individual filters of the hidden layers. So let's get to it:

In [ ]:
# Aggregate the layers
layer_dict = dict([(layer.name, layer) for layer in model.layers])

In [ ]:
from tensorflow.keras import backend as K

# Specify a function that visualized the layers
def show_activation(layer_name):
    
    layer_output = layer_dict[layer_name].output

    fn = K.function([model.input], [layer_output])
    
    inp = X_train[0:1]
    
    this_hidden = fn([inp])[0]
    
    # plot the activations, 8 filters per row
    plt.figure(figsize=(16,8))
    nFilters = this_hidden.shape[-1]
    nColumn = 8 if nFilters >= 8 else nFilters
    for i in range(nFilters):
        plt.subplot(nFilters / nColumn, nColumn, i+1)
        plt.imshow(this_hidden[0,:,:,i], cmap='magma', interpolation='nearest')
        plt.axis('off')
    
    return 

Now we can plot the filters of the hidden layers:

In [ ]:
layer_dict

In [ ]:
show_activation('conv2d_3')

In [ ]:
show_activation('conv2d_4')

## Conclusion of 2D example

The classification of the training set gets incredibly high, while the validation set also reaches a reasonable accuracy level above 80. Nonetheless, by only investigating a slab of our fMRI dataset, we might have missed out on some important additional parameters.

An alternative solution might be to use 3D convoluted neural networks. But keep in mind that they will have even more parameters and probably take much longer to fit the model to the training data. Having said so, let's get to it.

## Super secret fancy surprise

Going back to `graphics`, `outputs` and `interactive instances` of `jupyter notebook`, we can even go crazier and actually include a running `tensorboard` instance to enable `interactive evaluation` of our model (the future is now):

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs